In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import cv2

import glob


# example of zoom image augmentation
from numpy import expand_dims
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot

In [2]:
main_path = R"D:\Imbalanced Dataset\chest_xray"

In [3]:
train_path = os.path.join(main_path,"train")
test_path=os.path.join(main_path,"test")
val_path=os.path.join(main_path,"val")



In [4]:
pneumonia_train_images = glob.glob(train_path+"\\PNEUMONIA\\*.jpeg")
normal_train_images = glob.glob(train_path+"\\NORMAL\\*.jpeg")

In [5]:
pneumonia_test_images = glob.glob(test_path+"\\PNEUMONIA\\*.jpeg")
normal_test_images = glob.glob(test_path+"\\NORMAL\\*.jpeg")

In [6]:
pneumonia_val_images = glob.glob(val_path+"\\PNEUMONIA\\*.jpeg")
normal_val_images = glob.glob(val_path+"\\NORMAL\\*.jpeg")

In [7]:
len(normal_train_images)

1341

In [8]:
len(pneumonia_train_images)

3875

In [9]:
data = pd.DataFrame(np.concatenate([[0]*len(normal_train_images) , [1] *  len(pneumonia_train_images)]),columns=["class"])

# Defining Image Generators

## Load image into array

In [10]:
Normal_train=[]
pneumonia_trian =[]

In [11]:
import cv2
from tqdm import tqdm
for dirs in tqdm(normal_train_images):
    img = cv2.imread(dirs)    
    img2gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    resized = cv2.resize(img2gray, (200,200))
    #print(resized.shape)
    Normal_train.append(np.array(resized))

100%|██████████| 1341/1341 [00:36<00:00, 37.12it/s]


In [12]:
import cv2
from tqdm import tqdm
for dirs in tqdm(pneumonia_train_images):
    img = cv2.imread(dirs)    
    img2gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    resized = cv2.resize(img2gray, (200,200))
    #print(resized.shape)
    pneumonia_trian.append(np.array(resized))

100%|██████████| 3875/3875 [00:37<00:00, 102.26it/s]


In [13]:
## Generating

In [14]:
Normal_train_array=[]

In [15]:
for i in tqdm(range(0,len(Normal_train))):
    #print(a.shape)
    #print(Normal_train[i].shape)
    Normal_train_array.append(Normal_train[i])

100%|██████████| 1341/1341 [00:00<00:00, 1343660.22it/s]


In [16]:
ind = 0
from tqdm import tqdm
for dirs in tqdm(normal_train_images):
    ind=ind+1
    img = cv2.imread(dirs) 
    data = img_to_array(img)
    samples = expand_dims(data, 0)
    
    if ind%2 ==0:
        continue
    else:
        datagen = ImageDataGenerator(zoom_range=[0.8,1.0])
        it = datagen.flow(samples, batch_size=1)
        for i in range(2):
            batch = it.next()
            image = batch[0].astype('uint8')
            resized = cv2.resize(image, (200,200))
            img2gray = cv2.cvtColor(resized, cv2.COLOR_RGB2GRAY)
            Normal_train_array.append(img2gray)

100%|██████████| 1341/1341 [11:01<00:00,  2.03it/s]


In [17]:
Normal_train_array_X = np.array(Normal_train_array)

In [18]:
Normal_train_array_y = [0]*len(Normal_train_array_X)

In [19]:
pneumonia_trian_array_X = np.array(pneumonia_trian)

In [20]:
pneumonia_train_array_y = [1]*len(pneumonia_trian_array_X)

In [21]:
import random

### Read testing Image

In [22]:
Normal_test=[]
pneumonia_test =[]
Normal_test_array=[]

In [23]:
import cv2
from tqdm import tqdm
for dirs in tqdm(normal_test_images):
    img = cv2.imread(dirs)    
    img2gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    resized = cv2.resize(img2gray, (200,200))
    #print(resized.shape)
    Normal_test.append(np.array(resized))

100%|██████████| 234/234 [00:04<00:00, 48.69it/s]


In [24]:
import cv2
from tqdm import tqdm
for dirs in tqdm(pneumonia_test_images):
    img = cv2.imread(dirs)    
    img2gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    resized = cv2.resize(img2gray, (200,200))
    #print(resized.shape)
    pneumonia_test.append(np.array(resized))

100%|██████████| 390/390 [00:03<00:00, 129.61it/s]


In [25]:
for i in tqdm(range(0,len(Normal_test))):
    #print(a.shape)
    #print(Normal_train[i].shape)
    Normal_test_array.append(Normal_test[i])

100%|██████████| 234/234 [00:00<00:00, 234520.22it/s]


In [26]:
Normal_test_array_X = np.array(Normal_test_array)

In [27]:
Normal_test_array_y = [0]*len(Normal_test_array_X)

In [28]:
pneumonia_test_array_X = np.array(pneumonia_test)

In [29]:
pneumonia_test_array_y = [1]*len(pneumonia_test_array_X)

In [30]:
X_test = np.concatenate((Normal_test_array_X, pneumonia_test_array_X), axis=0)

In [31]:
y_test = np.concatenate((np.array(Normal_test_array_y), np.array(pneumonia_test_array_y)), axis=0)

In [32]:
y_test.shape

(624,)

In [33]:
X_test.shape

(624, 200, 200)

# Start fitting

In [34]:
from tensorflow.keras.callbacks import LearningRateScheduler

# This is a sample of a scheduler I used in the past
def lr_scheduler(epoch, lr):
    if epoch % 100 == 0 and epoch:
        lr = lr - lr*0.1
        return lr
    return lr

In [35]:
# example of loading the vgg16 model
from tensorflow.keras.applications import ResNet50V2, DenseNet121
from tensorflow.keras.layers import Input, Flatten, Dense
import tensorflow

def training(X, y, index, epoch):
    # define the keras model
    model_path = 'DL_model_Bagging/model_'+str(index)+'.h5'
    
    X = X.reshape(X.shape[0],X.shape[1], X.shape[2], 1)
    
    model = ResNet50V2(include_top=False, weights=None, input_tensor=Input(shape=((X.shape[1], X.shape[2], 1))))
    
    x = model.output
    x = Flatten()(x)
    x = Dense(64, activation = 'sigmoid')(x)
    x = Dense(2, activation = "sigmoid")(x)
    model = tensorflow.keras.Model(model.input, x)
    # summarize the model
    #model.summary()   
    
    
    
    
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.fit(X, y, epochs=epoch, batch_size=120, validation_split=0.2,verbose=1, 
                       callbacks = [LearningRateScheduler(lr_scheduler, verbose=0),
                                    tensorflow.keras.callbacks.ModelCheckpoint(model_path,
                                                                               monitor='val_loss',
                                                                               save_best_only=True,
                                                                               mode='min',
                                                                               verbose=0)])

In [36]:
from sklearn.metrics import confusion_matrix

    
import math
def conf_matrix(y_real, y_predict):
    tn, fp, fn, tp = confusion_matrix(y_real, y_predict).ravel()

    #print('confusion matrix: (tn, fp, fn, tp)',tn, fp, fn, tp)
    acc= (tp + tn)/(tp+tn+fp+fn)
    pre= tp/(tp+fp)
    sen= (tp)/(tp+fn)
    spec= (tn)/(tn+fp)
    F1score= 2*(tp)/(2*tp+fp+fn)  
    mcc= ((tp*tn)-(fp*fn))/math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))

    return tn, fp, fn, tp, acc, pre, sen, spec, F1score, mcc

In [37]:
from tensorflow.keras.utils import to_categorical
def one_hot(y):
    return to_categorical(y)

In [38]:
from tensorflow.keras.models import load_model

def model_Predict(X):
    try:
        del(model)
    except:
        print("Oops!  Nomodel to delete...")
    
    
   
    X = X.reshape(X.shape[0],X.shape[1], X.shape[2], 1)
    
    RM_predicted =[]
    for i in range(0, num_Interation):
        model = load_model('DL_model_Bagging/model_'+str(i)+".h5")
        RM_predicted.append(model.predict(X))
    
        
    return RM_predicted

## Checking on Time Calculation

In [40]:
X_train_List[i].shape

NameError: name 'X_train_List' is not defined

In [41]:
to_categorical(y_train_List[i]).shape

NameError: name 'y_train_List' is not defined

In [42]:
from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.utils import to_categorical
# define undersample strategy
undersample = RandomUnderSampler(sampling_strategy='majority')
from tqdm import tqdm
import warnings
import timeit
import tracemalloc
warnings.filterwarnings('ignore')
# split 10 fold of undersampling
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve
from tensorflow.keras import layers, Model


max_auc = 0.88

for time in range(0, 1):
    rsFile = "Result_OverIteration_Bagging_Numeric/Image/TimeCalculation"+str(time)+".csv"
    f2=open(rsFile,"w")
    f2.write('Time_train, Train_Peak_RAM, Time_predict, Peak_Time_prediction, thres, tn, fp, fn, tp, acc, pre, Recall, spec, F1score, mcc, auc\n')
    for num_Interation in ([15]):
        
        for epoch in ([10]):


            print("working on num_Interation: "+str(num_Interation) +"With epoch: "+str(epoch) )
            X_train_List=[]
            y_train_List = []

            
            
            
            #Generating the bags
    
            for i in range(0, num_Interation):
                #print(len(Normal_train_array)*1.3)
                pneumonia = np.array(random.sample(pneumonia_trian, int(len(Normal_train_array)*1.41)))
                normal = np.array(Normal_train_array)
                X_train_List.append(np.concatenate((normal, pneumonia)))
                
                
                y_train_List.append(np.concatenate((([0]*len(normal), [1]*(len(pneumonia))))))

            X_train_List = np.array(X_train_List)
            y_train_List = np.array(y_train_List)

            X_train_List = np.array(X_train_List)
            y_train_List = np.array(y_train_List)


            # ============== Starting training
            start = timeit.default_timer()
            tracemalloc.start()
            
            ## Start training
            for i in range(0, num_Interation):
                training(X_train_List[i], to_categorical(y_train_List[i]) , i, epoch)
                
            current, peak = tracemalloc.get_traced_memory()
            stop = timeit.default_timer()
            Time_train=stop - start
            Train_Peak_RAM = peak / 10**6

            
            
            start = timeit.default_timer()
            tracemalloc.start()
            
            predicted = model_Predict(X_test)
            stop = timeit.default_timer()
            Time_predict=stop - start
            current, peak = tracemalloc.get_traced_memory()
            Peak_Time_prediction = peak / 10**6
            
            
            result = np.array([[0]*predicted[i].shape[1]]*predicted[i].shape[0])

            for i in range(0, num_Interation):
                #print(result)
                result = result+ predicted[i]
                ##print('*=================')




            #print(y_test)
            #print(len(result[:, 1]))


            auc=roc_auc_score(y_test, result[:, 1]/num_Interation)
            #print(np.argmax(y_test, axis=1))
            print('==========================================')
            #print(result[:, 1]/num_Interation)
            #print(result[:, 1].shape)

            if max_auc <= auc:
                max_auc = auc
                print('======= auc =======: ', auc)




            print("result: ", result)
            print("result[:, 1].min() : ", result[:, 1].min())
            print("result[:, 1].max() : ", result[:, 1].max())
            thres = result[:, 1].min()
            while (thres <= result[:, 1].max()):
            #for thres in range(1, 1001):
                thres = thres+(result[:, 1].max()/500)
                yhat = []
                for i in range(len(result)):
                    if result[i][1]>thres:
                        yhat.append(1)
                    else:
                        yhat.append(0)

                yhat = np.array(yhat)

                tn, fp, fn, tp, acc, pre, Recall, spec, F1score, mcc = conf_matrix(y_test, yhat)
                #auc = roc_plot(np.argmax(y_test, axis=1), yhat)
                f2.write(str(Time_train)+", "+str(Train_Peak_RAM)+", "+str(Time_predict)+", "+str(Peak_Time_prediction)+", "+str(thres)+", "+str(tn)+", "+str(fp)+", "+str(fn)+", "+str(tp)+", "+str(acc)+", "+str(pre)+", "+str(Recall)+", "+str(spec)+", "+str(F1score)+", "+str(mcc)+", "+str(auc)+"\n")


    f2.close()
    print('WRITING FILE SUCESSFULL ========!!!!!!!!!!!!!!!!!!!!!!')       

working on num_Interation: 15With epoch: 10
Train on 5172 samples, validate on 1294 samples
Epoch 1/10
5172/5172 [==============================] - 54s 10ms/sample - loss: 0.4482 - accuracy: 0.8406 - val_loss: 1.7334 - val_accuracy: 0.0054
Epoch 2/10
5172/5172 [==============================] - 27s 5ms/sample - loss: 0.2505 - accuracy: 0.9319 - val_loss: 0.3150 - val_accuracy: 0.9324
Epoch 3/10
5172/5172 [==============================] - 26s 5ms/sample - loss: 0.2096 - accuracy: 0.9414 - val_loss: 0.4078 - val_accuracy: 0.8211
Epoch 4/10
5172/5172 [==============================] - 26s 5ms/sample - loss: 0.1576 - accuracy: 0.9569 - val_loss: 2.4887 - val_accuracy: 0.0000e+00
Epoch 5/10
5172/5172 [==============================] - 26s 5ms/sample - loss: 0.1560 - accuracy: 0.9564 - val_loss: 2.0789 - val_accuracy: 0.0641
Epoch 6/10
5172/5172 [==============================] - 26s 5ms/sample - loss: 0.1344 - accuracy: 0.9592 - val_loss: 2.0780 - val_accuracy: 0.2689
Epoch 7/10
5172/5172 

In [ ]:
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt


prediction = np.argmax(result, axis=1) 

confmat = confusion_matrix(y_true=y_test, y_pred=prediction)
fig, ax = plt.subplots(figsize=(3, 3))
ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat.shape[0]):
    for j in range(confmat.shape[1]):
        ax.text(x=j, y=i, s=confmat[i,j], va='center', ha='center')
plt.xlabel('predicted label')        
plt.ylabel('true label')
plt.show()

conf_matrix(y_test, prediction)
roc_plot(y_test, prediction)

In [ ]:
result

In [ ]:
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt

for thres in range(1, 1001):
    print('thres: ', thres/100)
    yhat = []
    thres=thres/100
    for i in range(len(result)):
        if result[i][0]>thres:
            yhat.append(0)
        else:
            yhat.append(1)

    yhat = np.array(yhat)

    confmat = confusion_matrix(y_true=y_test, y_pred=yhat)
    fig, ax = plt.subplots(figsize=(3, 3))
    ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
    for i in range(confmat.shape[0]):
        for j in range(confmat.shape[1]):
            ax.text(x=j, y=i, s=confmat[i,j], va='center', ha='center')
    plt.xlabel('predicted label')        
    plt.ylabel('true label')
    plt.show()

    conf_matrix(y_test, yhat)
    roc_plot(y_test, yhat)